## Import

In [1]:
!pip install xgboost
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 MB 52.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.8/296.8 MB 44.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [xgboost]m1/2 [xgboost]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 49.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [catboost]1/2 [catboost]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
#import and connect to service
#所有注释带“？”的是今天新加的注释
import bql
import pandas as pd
import numpy as np
import datetime as dt
import calendar
from typing import List
from scipy.stats import mstats

bq = bql.Service()

### Generate rolling member

In [7]:
# 在 BQuant 笔记本里运行：SPX 指数季度调整（第三个周五后生效）成分股清单
# === 1) 生成每年 3/6/9/12 的“第三个星期五后生效”的 as-of 日期 ===
def third_friday(year:int, month:int) -> pd.Timestamp:
    cal = calendar.monthcalendar(year, month)
    fridays = [week[calendar.FRIDAY] for week in cal if week[calendar.FRIDAY] != 0]
    day = fridays[2]  # 第三个星期五
    return pd.Timestamp(year=year, month=month, day=day)

def effective_next_business_day(ts: pd.Timestamp) -> pd.Timestamp:
    # “后生效”通常为下一个工作日（周一）。这里用 BusinessDay(1) 近似（忽略美股节假日）。
    # 若你有美股交易日历，可替换为自有日历的下一个交易日。
    return ts + pd.offsets.BDay(1)

years = range(2019, 2026)  # 2019~2025？
q_months = [3, 6, 9, 12]

asof_list = []
for y in years:
    for m in q_months:
        if y == 2019 and m < 12:
            continue  # 从 2019-12 起
        elif y == 2025 and m > 9:
            continue
        tfr = third_friday(y, m)
        eff = effective_next_business_day(tfr)
        asof_list.append(eff)

asof_list = sorted(set(asof_list))
asof_strs = [d.strftime("%Y-%m-%d") for d in asof_list]
print("将查询这些 as-of 生效日：", asof_strs)

# === 2) 定义指数成员 Universe 与数据项（对象式 API，避免 combined_df） ===
univ = bq.univ.members("SPX Index")
items = {
    "ID":   bq.data.id(),
    "NAME": bq.data.name(),
}

# === 3) 逐个 as-of 拉取成分股，并做列名/去重/过滤 ===
frames = []
for asof in asof_strs:
    req = bql.Request(univ, items, with_params={"asofdate": asof, "mode": "cached"})#这里用了asofdate？
    res = bq.execute(req)

    # 不用 combined_df：逐个结果列拼接，避免弃用与列冲突问题
    cols = []
    for r in res:
        df_r = r.df()
        s = df_r[r.name]          # 取该数据项的 Series
        s.name = r.name           # 列名设回 'ID' 或 'NAME'
        cols.append(s)

    df = pd.concat(cols, axis=1).reset_index(drop=True)

    # 统一到我们习惯的列名
    rename_map = {}
    for c in df.columns:
        lc = str(c).lower()
        if lc == "id":
            rename_map[c] = "ID"
        elif lc == "name":
            rename_map[c] = "security_name"
    df = df.rename(columns=rename_map)

    # 兜底处理：确保有 ticker 列
    if "ID" not in df.columns:
        df = df.rename(columns={df.columns[0]: "ID"})
    if "security_name" not in df.columns:
        df["security_name"] = None

    # 过滤异常：去掉指数本体；尽量保留股票证券（常见后缀 "Equity"）
    df = df[~df["ID"].astype(str).str.endswith(" Index", na=False)]
    df = df[df["ID"].astype(str).str.contains("Equity", na=False)]

    df = df.drop_duplicates(subset=["ID"]).assign(DATE=asof)
    frames.append(df[["DATE", "ID", "security_name"]])

# === 4) 合并与基本校验 ===
out = (
    pd.concat(frames, ignore_index=True)
      .sort_values(["DATE", "ID"])
      .reset_index(drop=True)
)

# 每个生效日大致应接近 500 只
check = out.groupby("DATE")["ID"].nunique().rename("n_constituents")
print(check)

# 预览与导出
out.head(10)
# out.to_csv("SPX_members_quarterly_effective_201912_202412.csv", index=False)
out.to_parquet("SPX_members_quarterly_201912_202509.parquet", index=False)

将查询这些 as-of 生效日： ['2019-12-23', '2020-03-23', '2020-06-22', '2020-09-21', '2020-12-21', '2021-03-22', '2021-06-21', '2021-09-20', '2021-12-20', '2022-03-21', '2022-06-20', '2022-09-19', '2022-12-19', '2023-03-20', '2023-06-19', '2023-09-18', '2023-12-18', '2024-03-18', '2024-06-24', '2024-09-23', '2024-12-23', '2025-03-24', '2025-06-23', '2025-09-22']
DATE
2019-12-23    503
2020-03-23    503
2020-06-22    503
2020-09-21    503
2020-12-21    503
2021-03-22    503
2021-06-21    503
2021-09-20    503
2021-12-20    503
2022-03-21    503
2022-06-20    503
2022-09-19    503
2022-12-19    503
2023-03-20    503
2023-06-19    503
2023-09-18    503
2023-12-18    503
2024-03-18    503
2024-06-24    503
2024-09-23    503
2024-12-23    503
2025-03-24    503
2025-06-23    503
2025-09-22    503
Name: n_constituents, dtype: int64


In [8]:
out.to_csv("SPX_members_quarterly_201912_202509.csv", index=False)

### Generate rolling member price

In [16]:
# ===========================================
# BQuant：读取附件成分股 → 拉取OHLCV → 输出CSV
# 覆盖范围：2020-01-01 ~ 2024-12-31
# 输出文件：spx_ohlcv_20200101_20241231.csv
# 说明：
#   1) 优先使用 get_time_series；失败则自动回退对象式 API（支持分批）。
#   2) 列名统一为：ts_code, trade_date, open, high, low, close, vol
#   3) 仅做OHLCV，不含技术指标。
# ===========================================
# ---- 参数 ----
START = "2019-12-23"
END   = "2025-09-22"
MEMBERS_PATHS = [
    "SPX_members_quarterly_201912_202509.csv"  # 你的附件默认路径
]
OUT_CSV = "spx_ohlcv_201912_202509.csv"
# ---- 读取成分股 ----
members = None
for p in MEMBERS_PATHS:
    try:
        df = pd.read_csv(p)
        if "ID" not in df.columns:
            df = df.rename(columns={df.columns[0]: "ID"})
        # 常见清洗：去指数本体/保留 Equity
        df = df[~df["ID"].astype(str).str.endswith("Index", na=False)]
        if df["ID"].astype(str).str.contains("Equity", na=False).any():
            df = df[df["ID"].astype(str).str.contains("Equity", na=False)]
        members = df
        print(f"[OK] 读取成分股：{p}，行数={len(df)}")
        break
    except Exception as e:
        print(f"[WARN] 未能读取 {p}: {repr(e)}")

if members is None:
    raise FileNotFoundError("未找到成分股附件，请检查 MEMBERS_PATHS 路径或文件名。")

tickers: List[str] = sorted(pd.Series(members["ID"].astype(str).unique()).dropna().tolist())
print(f"[INFO] 去重后股票数：{len(tickers)}")

# ---- 拉取 OHLCV ----
def fetch_ohlcv(tickers: List[str], start: str, end: str) -> pd.DataFrame:
    """
    返回列：ID, DATE, close
    """
    # 方案A：get_time_series（更简洁）
    try:
        from bql.util import get_time_series
        fields = ["px_last"]#can change fields here
        data = get_time_series(tickers, fields, start, end)
        #这里的列命名也许需要修改？
        df = (
            data.reset_index()
                .rename(columns={
                    "ID": "ID", "DATE": "DATE",
                    "px_last": "close"
                })
        )
        print("[OK] get_time_series 成功。")
        return df[["ID", "DATE", "close"]]
    except Exception as e:
        print(f"[INFO] get_time_series 不可用，切换对象式 API：{repr(e)}")

    # 方案B：对象式 API + dates=range（分批）
    frames = []
    rng = bq.func.range(start, end)
    BATCH = 120  # 视资源配额/性能可调大/调小
    for i in range(0, len(tickers), BATCH):
        batch = tickers[i:i+BATCH]
        univ = bq.univ.list(batch)
        items = {
            "CLOSE": bq.data.px_last(dates=rng),
        }
        req = bql.Request(univ, items)  # 如需快照模式：with_params={"mode": "cached"}
        res = bq.execute(req)

        # 将每个字段结果拼成宽表
        parts = []
        for r in res:
            t = r.df().reset_index()  # 包含 ID, DATE, r.name
            t = t.rename(columns={"ID": "ID", "DATE": "DATE", r.name: r.name.lower()})
            parts.append(t[["ID", "DATE", r.name.lower()]])
        df_merged = parts[0]
        for p in parts[1:]:
            df_merged = df_merged.merge(p, on=["ID", "DATE"], how="outer")
        frames.append(df_merged)
        print(f"[PROG] 批次 {i//BATCH+1}: 累计股票 {min(i+BATCH, len(tickers))}/{len(tickers)}")

    df = pd.concat(frames, ignore_index=True)
    df = df.rename(columns={"close": "close"})
    return df[["ID", "DATE", "close"]]

ohlcv = fetch_ohlcv(tickers, START, END)

# ---- 整理与导出 ----
ohlcv["DATE"] = pd.to_datetime(ohlcv["DATE"])
ohlcv = (
    ohlcv.dropna(subset=["ID", "DATE"])
         .sort_values(["ID", "DATE"])
         .reset_index(drop=True)
)
print(ohlcv.head())

ohlcv.to_csv(OUT_CSV, index=False, encoding="utf-8")
ohlcv.to_parquet("spx_ohlcv_201912_202509.parquet", index=False)
print(f"[DONE] 已写出：{OUT_CSV}，行数={len(ohlcv)}")

[OK] 读取成分股：SPX_members_quarterly_201912_202509.csv，行数=12072
[INFO] 去重后股票数：503


[PendingDeprecationWarning] The 'combined_df' function is deprecated and may be removed in a future release. Please see https://help.bquant.blpprofessional.com/content?id=giZBGYBMGPMUX5YQdaWkqR&view=enterprise#bql.combined_df for recommended migration options.


[OK] get_time_series 成功。
            ID       DATE  close
0  A UN Equity 2019-12-23  85.26
1  A UN Equity 2019-12-24  85.30
2  A UN Equity 2019-12-26  85.45
3  A UN Equity 2019-12-27  85.42
4  A UN Equity 2019-12-30  84.90
[DONE] 已写出：spx_ohlcv_201912_202509.csv，行数=704577


### Generate index

In [17]:
def normalize_bql_dates(df):
    """
    Clean Bloomberg BQL dataframe:
    - unify DATE / AS_OF_DATE / REVISION_DATE / PERIOD_END_DATE → 'DATE'
    - drop duplicates and sort chronologically
    """
    df = df.copy()
    if 'DATE' not in df.columns:
        # pick the first date-like column
        for c in ['AS_OF_DATE', 'REVISION_DATE', 'PERIOD_END_DATE']:
            if c in df.columns:
                df['DATE'] = df[c]
                break
    # remove others
    for c in ['AS_OF_DATE', 'REVISION_DATE', 'PERIOD_END_DATE', 'CURRENCY']:
        if c in df.columns and c != 'DATE':
            df = df.drop(columns=c)
    # df = df.drop_duplicates(subset=['ID', 'DATE']).sort_values(['ID', 'DATE'])
    return df

In [34]:
dts = bq.func.range(START, END)
price_close = bq.data.px_last(dates=dts) #, fill="prev"

close = ohlcv.pivot(index="DATE",columns="ID",values="close").sort_index()

# Benchmark SPX
spx = bq.execute(bql.Request("SPX Index", price_close))[0].df().dropna()
spx = spx.reset_index()
cols = [c for c in spx.columns if 'PX_LAST' in c]
spx = spx.rename(columns={cols[0]: 'PX_LAST'})
spx_close = spx.pivot(index="DATE",columns="ID",values="PX_LAST")#.iloc[:,0]
spx_close.to_csv("spx_index.csv", encoding="utf-8")
spx_close.to_parquet("spx_index.parquet", index=False)

## Calculate Factor

### Fundamental factors

In [36]:
### ============== Fundamentals ========================
factors = {
    'EPS':bq.data.is_eps(dates=dts, fill="prev"), # earn_yld
    'PB':bq.data.px_to_book_ratio(dates=dts, fill="prev"),
    'PE':bq.data.pe_ratio(dates=dts, fill="prev"),
    'PS':bq.data.px_to_sales_ratio(dates=dts, fill="prev"),
    'ROE':bq.data.return_com_eqy(dates=dts, fill="prev"),
    'ROA':bq.data.return_on_asset(dates=dts, fill="prev"),
    'OM':bq.data.oper_margin(dates=dts, fill="prev"),
    'TOratio':bq.data.px_volume(dates=dts, fill="prev")/bq.data.eqy_sh_out(dates=dts, fill="prev"), # = vol/TotShares = turnover/MV
    'FFratio':bq.data.eqy_free_float_pct(dates=dts, fill='prev')/100.0,
    'AssetTO':bq.data.asset_turnover(dates=dts, fill="prev"),
    'Rating':bq.data.best_analyst_rating(dates=dts, fill="prev"),
}

# req2 = bql.Request(uni, factors)
# res2 = bq.execute(req2)
# meta_normed = [normalize_bql_dates(r.df()) for r in res2]

meta={}
for i,(name,f) in enumerate(factors.items()): 
    res = bq.execute(bql.Request(univ, factors[name]))
    res_date_normed = normalize_bql_dates(res[0].df())
    d = res_date_normed.reset_index()
    d_wide = d.pivot(index='DATE', columns='ID').sort_index()
    d_wide.columns = d_wide.columns.get_level_values(1)
    d_wide = d_wide.reindex(calendar)#.ffill(limit=60)
    meta[name] = d_wide


### Basic factor

In [41]:
# ======= Other factors ========
# 'VOL30':bq.data.volatility_30d_calc(dates=dts, fill="prev"),
# 'VOL90':bq.data.volatility_90d_calc(dates=dts, fill="prev"),
# 'EV_EBITDAR':bq.data.ev_to_ebitdar(dates=dts, fill="prev"), --> many NaN values
    # "IMB_PX_BUY":   bq.data.px_imbalance_buy(),
    # "IMB_PX_SELL":  bq.data.px_imbalance_sell(),
    # "IMB_VOL_BUY":  bq.data.px_order_imb_buy_volume(),
    # "IMB_VOL_SELL": bq.data.px_order_imb_sell_volume(),
    # "vwap_on_exchange":    bq.data.vwap_on_exchange(dates=dts),
    # "vwap_volume":         bq.data.vwap_volume(dates=dts),
    # "vwap_num_trades":     bq.data.vwap_num_trades(dates=dts),
    # "OFF_ON_VOL":  bq.data.off_on_exch_volume(dates=dts),
    # "BLOCK_ratio": bq.data.block_trade_accum_volume(dates=dts)/bq.data.px_volume(dates=dts), # block trade activity

# ======= Technicals ========
fields = {
    'OPEN':bq.data.px_open(dates=dts, fill="prev"),
    'HIGH':bq.data.px_high(dates=dts, fill="prev"),
    'LOW':bq.data.px_low(dates=dts, fill="prev"),
    'BID_ASK_SPR':bq.data.average_bid_ask_spread_pct(dates=dts, fill="prev"), #(ask-bid)/mid
    "VWAP": bq.data.vwap(dates=dts, fill="prev"),
    "VWAP_TOratio": bq.data.vwap_turnover(dates=dts)/bq.data.cur_mkt_cap(dates=dts),
    "VWAP_BID_VOL": bq.data.vwap_bid_vol_percentage(dates=dts),
    "VWAP_ASK_VOL": bq.data.vwap_ask_vol_percentage(dates=dts),
}

tech={}
for i,(name,f) in enumerate(fields.items()): 
    res = bq.execute(bql.Request(univ, fields[name]))
    res_date_normed = normalize_bql_dates(res[0].df())
    d = res_date_normed.reset_index()
    # d = d.rename(columns={df.columns[-1]: name})
    d_wide = d.pivot(index='DATE', columns='ID').sort_index()
    d_wide.columns = d_wide.columns.get_level_values(1)
    d_wide = d_wide.reindex(calendar)#.ffill(limit=60)
    tech[name] = d_wide

### Momentum Factor

In [42]:
### ============== Technicals ========================
MOM_WIN = 252
RSI_WIN = 14
MA_LONG = 200
TD_SETUP = 9
REV_WIN = 21
SKIP = 21

# === Returns ===
tech['DailyRet'] = close.pct_change()

# === Momentum ===
tech['Momentum_12_1'] = tech['DailyRet'].rolling(MOM_WIN+SKIP).apply(
    lambda x: np.prod(1+x[-(MOM_WIN+SKIP):-SKIP]) - 1, raw=True)

# === Reversal ===
tech['Reversal_1M'] = -tech['DailyRet'].rolling(SKIP).apply(lambda x: np.prod(1+x)-1, raw=True)

# === Moving averages ===
ma = close.rolling(MA_LONG).mean()
tech['MACross'] = (close / ma) - 1

# === RSI ===
def rsi(series, window=14):
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    roll_up = up.ewm(alpha=1/window, adjust=False).mean()
    roll_down = down.ewm(alpha=1/window, adjust=False).mean()
    rs = roll_up / (roll_down + 1e-12)
    return 100 - (100 / (1 + rs))
tech['RSI14'] = close.apply(rsi)

# === MACD ===
def macd_hist(series, f=12, s=26, sig=9):
    ema_f = series.ewm(span=f, adjust=False).mean()
    ema_s = series.ewm(span=s, adjust=False).mean()
    macd = ema_f - ema_s
    signal = macd.ewm(span=sig, adjust=False).mean()
    return macd, signal
macd_line, signal_line = macd_hist(close)
tech['MACD'] = macd_line - signal_line
tech['MACD_diff'] = tech['MACD'].diff()

# === Volatility ===
tech['VOL20'] = tech['DailyRet'].rolling(20).std()
tech['VOL60'] = tech['DailyRet'].rolling(60).std()
# vol252 = ret.rolling(252).std()

# === Candlestick data ===
upper_shadow = tech['HIGH'] - pd.DataFrame(np.maximum(tech['OPEN'], close))
lower_shadow = pd.DataFrame(np.minimum(tech['OPEN'], close)) - tech['LOW']
tech['Upp_Shad'] = upper_shadow / (tech['HIGH'] - tech['LOW'])
tech['Low_Shad'] = lower_shadow / (tech['HIGH'] - tech['LOW'])
tech['Body'] = abs(close - tech['OPEN']) / (tech['HIGH'] - tech['LOW'])

# Buy/sell pressure 
tech['BS_Pressure'] = (close - tech['VWAP']) / tech['VWAP'] # Positive means buying pressure; negative means selling pressure
# Price impact factor
avg_vol = vol.rolling(window=20, min_periods=5).mean()
tech['Price_Impact'] = (close - tech['OPEN'])/  tech['OPEN'] / (vol / avg_vol)
# Volume spread
tech['Volume_Spread'] = ((vol - vol.shift(1)) / vol.shift(1)) * np.sign(close - close.shift(1))
# Liquidity Strength
tech['Liquid_Strength'] = tech['BS_Pressure'] / (1 + tech['BID_ASK_SPR'])

# === Beta vs SPX ===
bench_ret = spx_close.pct_change().iloc[:, 0]
def rolling_beta(asset_ret, bench_ret, win=252):
    cov = asset_ret.rolling(win).cov(bench_ret)
    var = bench_ret.rolling(win).var()
    return cov / var
tech['Beta252'] = tech['DailyRet'].apply(lambda s: rolling_beta(s, bench_ret))


In [43]:
all_factors = {**meta, **tech}
all_factors['Close'] = close
combined = pd.concat(all_factors, axis=1)
combined_stacked = combined.stack(level=1).sort_values(['ID', 'DATE'])

combined_stacked.reset_index()

,DATE,ID,EPS,PB,PE,PS,ROE,ROA,OM,TOratio,...,VOL60,Upp_Shad,Low_Shad,Body,BS_Pressure,Price_Impact,Volume_Spread,Liquid_Strength,Beta252,Close
0,2019-12-23,A UN Equity,3.41,5.548724,31.328588,5.122262,22.995169,11.904630,18.225838,0.000846,...,NaN,0.032258,0.709677,0.258065,0.000027,NaN,NaN,0.000026,NaN,85.26
1,2019-12-24,A UN Equity,3.41,5.551327,31.343286,5.124665,22.995169,11.904630,18.225838,0.000307,...,NaN,0.000000,0.615385,0.384615,-0.000048,NaN,-0.636715,-0.000046,NaN,85.30
2,2019-12-26,A UN Equity,3.41,5.561089,31.398403,5.133677,22.995169,11.904630,18.225838,0.000692,...,NaN,0.323810,0.409524,0.266667,0.000639,NaN,1.252625,0.000623,NaN,85.45
3,2019-12-27,A UN Equity,3.41,5.559136,31.387379,5.131874,22.995169,11.904630,18.225838,0.000876,...,NaN,0.000000,0.539823,0.460177,0.000451,NaN,-0.265716,0.000439,NaN,85.42
4,2019-12-30,A UN Equity,3.41,5.525295,31.196307,5.100634,22.995169,11.904630,18.225838,0.000742,...,NaN,0.000000,0.343284,0.656716,-0.000404,-0.004814,0.153265,-0.000392,NaN,84.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719937,2025-09-16,ZTS UN Equity,5.82,13.153349,24.044278,6.963435,52.539475,18.237676,37.112413,0.001408,...,0.012436,0.465753,0.000000,0.534247,0.000483,0.008010,0.042297,0.000448,0.552967,147.46
719938,2025-09-17,ZTS UN Equity,5.82,13.106073,23.957858,6.938407,52.539475,18.237676,37.112413,0.001185,...,0.012438,0.727586,0.110345,0.162069,-0.001331,-0.003766,0.158513,-0.001198,0.553011,146.93
719939,2025-09-18,ZTS UN Equity,5.82,13.121237,23.985578,6.946435,52.539475,18.237676,37.112413,0.001482,...,0.012431,0.298561,0.413669,0.287770,0.000314,0.005137,0.250905,0.000290,0.552724,147.10
719940,2025-09-19,ZTS UN Equity,5.82,13.012414,23.786649,6.888823,52.539475,18.237676,37.112413,0.004780,...,0.012463,0.000000,0.246575,0.753425,-0.001993,-0.003673,-2.224675,-0.001822,0.551698,145.88


In [ ]:
combined_stacked.to_csv("sp500_all_factors.csv", index=False, encoding="utf-8")
combined_stacked.to_parquet("sp500_all_factors.parquet", index=False, compression='snappy')